# Web Scraping: EveryNoise.com

source: [http://everynoise.com/everynoise1d.cgi?scope=all]



In [48]:
from bs4 import BeautifulSoup
import requests, sys
from collections import defaultdict
import pickle as pickle
from tqdm import tqdm

In [5]:
## Request HTML

r = requests.get('http://everynoise.com/everynoise1d.cgi?vector=popularity&scope=all')

In [33]:
## Parse Genre HTML Elements
soup = BeautifulSoup(r.text, "html.parser")
table = soup.body.table
elements = table.find_all('td', 'note')
genres = []
for i, row in enumerate(elements):
    if i % 2:
        genres.append(row.contents[0].contents[0])
print("Total genres:", len(genres))
print("First ten genres:\n", genres[:10])

Total genres: 5071
First ten genres:
 ['pop', 'dance pop', 'rap', 'pop dance', 'pop rap', 'rock', 'post-teen pop', 'latin', 'hip hop', 'trap']


In [49]:
%%time

## Request Playlist HTMLs
spotify_links = dict()

for i in tqdm(range(len(genres)), desc="Fetching..."):
        playlist_r = requests.get('http://everynoise.com/everynoise1d.cgi?root=${genres[i]}&scope=all')
        soup = BeautifulSoup(playlist_r.text, "html.parser")
        playlist_link = soup.find_all(id='spotifylink')
        spotify_links[genres[i]] = playlist_link[0]['href']

print('Total links:', len(spotify_links))

Fetching...: 100%|██████████| 5071/5071 [4:03:01<00:00,  2.88s/it]Total links: 5071



TypeError: 'dict_items' object is not subscriptable

In [53]:
import json

with open('../data/spotify_links.json', 'w') as outfile:
    json.dump({'links': spotify_links}, outfile, indent=4)